In [1]:
# Thoomas Goldman 2023
# DOLPHINN

import sys
import os

#Get the absolute path of the current script
current_path = os.path.dirname(os.path.abspath('__file__'))
dolphinn_path = os.path.join(current_path, '..')
sys.path.append(dolphinn_path)

from DOLPHINN.pinn import DOLPHINN
from DOLPHINN.dynamics import TwoBodyProblemRadialNonDimensionalControl
from DOLPHINN.output_layers import InitialFinalStateLayer_Radial
from DOLPHINN.training import Scheduler, Restarter
from DOLPHINN.objectives import OptimalFuel

import numpy as np

Using backend: tensorflow
Other supported backends: tensorflow.compat.v1, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
mu = 1.32712440042e20 # SUN
m = 100 #Spacecraft mass
AU = 1.5e11 #EARTH-SUN
a = 10
umax = 1
isp = 2500

rp = AU
ra = 1.5*AU

e = (ra - rp)/(ra + rp)
sma = (rp + ra)/2

# Initial state
r0 = AU
vr0 = 0
vt0 = np.sqrt(mu/r0)
initial_state = np.array([r0, vr0, vt0])

# Final state
rfinal = 1.5*AU
vr_final = 0
vt_final = np.sqrt(mu/rfinal)
final_state = np.array([rfinal, vr_final, vt_final])

# Non dimensionalize
length_scale = AU
time_scale = AU / vt0

initial_state[:1] = initial_state[:1]/length_scale
initial_state[1:] = initial_state[1:]*time_scale/length_scale

final_state[:1] = final_state[:1]/length_scale
final_state[1:] = final_state[1:]*time_scale/length_scale

# Integration times
period = 2*np.pi*np.sqrt(sma**3/mu)
t0 = 0
tfinal = 1.5*period
N = 200

# Network architecture
input_dimension = 1
output_dimension = 5
neurons = 5
hidden_layers = 3
layer_size = [input_dimension] + [[neurons]*output_dimension] * hidden_layers + [output_dimension]

data = {"initial_state": initial_state,
        "final_state": final_state,
        "mu": mu,
        "m": m,
        "a": a,
        "umax": umax,
        "isp": isp,
        "length_scale": length_scale,
        "time_scale": time_scale,
        "t0": t0,
        "tfinal": tfinal/time_scale,
        "architecture": layer_size,
        "activation": "sin",
        "N_train": N,
        "N_boundary": 2,
        "N_test": N,
        "sampler": "perturbed_uniform_tf"}

In [3]:
# Create problem
problem = DOLPHINN(data,
                   TwoBodyProblemRadialNonDimensionalControl,
                   output_transform = InitialFinalStateLayer_Radial,
                   objective = OptimalFuel)

restarter = Restarter(loss_weigths = [1, 1, 1, 1e-7])
problem.train(restarter)

Using time-dependent random seed: 20230626192247
Initialisation attempt: 1
Using time-dependent random seed: 20230626192247
Compiling model...
'compile' took 0.002320 s

Training model...

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Step      Train loss                                  Test loss                                   Test metric
0         [2.29e+00, 4.59e+09, 3.07e+01, 8.35e-04]    [6.52e-01, 7.33e+07, 2.87e+01, 8.25e-04]    []  
1000      [1.10e+01, 9.91e+02, 2.52e+00, 3.35e-05]    [4.08e+00, 5.67e+00, 7.11e-01, 2.64e-05]    []  
2000      [1.38e+01, 9.68e+02, 2.75e+00, 4.04e-05]    [6.91e+00, 6.21e+00, 9.97e-01, 3.25e-05]    []  
3000      [1.30e+01, 9.56e+02, 2.59e+00, 5.28e-05]    [6.17e+00, 9.25e+00, 8.54e-01, 4.46e-05]    []  

Best model at step 3000:
  train loss: 9.72e+02
  test loss: 1.63e+01
  test metric: []

In [4]:
schedule = Scheduler([(1e-3, 3000),(1e-4, 10000)],
                     loss_weigths= [1, 1, 1, 1e-7]
                    )

problem.train(schedule)

Compiling model...
'compile' took 0.089337 s

Training model...

Step      Train loss                                  Test loss                                   Test metric
3000      [1.43e+00, 3.70e-02, 8.70e-03, 2.03e-05]    [2.10e-01, 3.71e-02, 8.18e-03, 1.57e-05]    []  
4000      [1.14e-04, 3.41e-05, 6.67e-05, 2.93e-06]    [1.08e-04, 3.39e-05, 6.76e-05, 2.48e-06]    []  
5000      [2.78e-05, 7.76e-06, 1.89e-05, 2.38e-06]    [2.60e-05, 6.92e-06, 1.92e-05, 2.09e-06]    []  
6000      [1.83e-05, 5.44e-06, 1.34e-05, 2.21e-06]    [1.73e-05, 5.12e-06, 9.07e-06, 1.96e-06]    []  

Best model at step 6000:
  train loss: 3.93e-05
  test loss: 3.34e-05
  test metric: []

'train' took 10.880631 s

Compiling model...
'compile' took 0.084991 s

Training model...

Step      Train loss                                  Test loss                                   Test metric
6000      [1.83e-05, 5.44e-06, 1.34e-05, 2.21e-06]    [1.73e-05, 5.12e-06, 9.07e-06, 1.96e-06]    []  
7000      [5.00e-06

In [5]:
problem.store("../Data/test/", overwrite = True)

Saving loss history to ../Data/test/loss.dat ...
Saving training data to ../Data/test/train.dat ...
Saving test data to ../Data/test/test.dat ...


In [6]:
problem2 = DOLPHINN.from_config("../Data/test/config", 
                               train = True)

Config file succesfully parsed. Initializing DOLPHINN with:

dynamics:    TwoBodyProblemRadialNonDimensionalControl
objective:    OptimalFuel
output_transform:    InitialFinalStateLayer_Radial
input_transform:    None
initial_state:    [1.0, 0.0, 1.0]
final_state:    [1.5, 0.0, 0.816496580927726]
mu:    1.32712440042e+20
m:    100
a:    10
umax:    1
isp:    2500
length_scale:    150000000000.0
time_scale:    5042908.305393074
t0:    0
tfinal:    13.17152762070136
architecture:    [1, [5, 5, 5, 5, 5], [5, 5, 5, 5, 5], [5, 5, 5, 5, 5], 5]
activation:    sin
N_train:    200
N_boundary:    2
N_test:    200
sampler:    perturbed_uniform_tf
seed:    20230626191839
train_1:    {'iterations': 3000, 'loss_threshold': 10, 'loss_weights': [1, 1, 1, 1e-07], 'lr': 0.01, 'max_attempts': 50, 'name': 'Restarter'}
train_2:    {'loss_weights': [1, 1, 1, 1e-07], 'name': 'Scheduler', 'schedule': [[0.001, 3000], [0.0001, 10000]]}


TypeError: Restarter.__init__() got an unexpected keyword argument 'loss_weights'

In [ ]:
problem2.